In [1]:
import torch
import numpy

행렬 곱 구현

In [2]:
x = torch.FloatTensor([[1,2],
                       [3,4],
                       [4,5]])

y = torch.FloatTensor([[1,2],
                       [1,2]])

print(x.size(), y.size())

z = torch.matmul(x,y)
print(z.size())

torch.Size([3, 2]) torch.Size([2, 2])


In [6]:
x = torch.FloatTensor(3, 3, 2)
y = torch.FloatTensor(3, 2, 3)
z = torch.bmm(x, y)
print(z.size())

torch.Size([3, 3, 3])


선형 계층 : 행렬 곱셈과 벡터의 덧셈으로 이루어져 있기 때문에 선형변환 이라 볼 수 있다. 

In [7]:
# 선형 계층 구현

# 3*2 크기의 행렬 W
W = torch.FloatTensor([[1,2],
                       [3,4],
                       [5,6]])

# 2개의 요소를 갖는 벡터 b
b = torch.FloatTensor([2,2])

def linear(x, W, b):
    y = torch.matmul(x, W)+b
    
    return y

x = torch.FloatTensor(4,3)
y = linear(x, W, b)
print(y.size())


torch.Size([4, 2])


torch.nn.Module 클래스 상속 받기 

In [9]:
# nn.Module 을 상속받은 클래스는 보통 __init__ 과 forward 사용
# __init__ : 계층 내부에서 필요한 변수를 미리 선언 + 또 다른 계층을 소유하도록 할 수 있음
# forward :  계층을 통과하는데 필요한 계산 수행 

import torch.nn as nn 

class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.W = torch.FloatTensor(input_dim, output_dim)
        self.b = torch.FloatTensor(output_dim)


    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)

        return y

linear = MyLinear(3,2)
y = linear(x)

# 아직 제대로 된 모델이 아님( 학습할 수 있는 파라미터가 없음 ) 
for p in linear.parameters():
    print(p)

올바른 방법:nn.Parameter 활용하기 

In [15]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()
        
        # 학습이 가능하도록 torch.nn.Parameter 클래스 활용 
        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) +self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)

        return y


for p in linear.parameters():
    print(p)


Parameter containing:
tensor([[ 0.1555, -0.3334, -0.5157],
        [-0.3579,  0.1233, -0.3723]], requires_grad=True)
Parameter containing:
tensor([-0.1968,  0.0085], requires_grad=True)


nn.Linear 활용하기 

In [30]:
linear = nn.Linear(3,2)
y = linear(x)

for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.2484,  0.2972, -0.4306],
        [-0.1090, -0.3679, -0.2190]], requires_grad=True)
Parameter containing:
tensor([0.2391, 0.3523], requires_grad=True)


In [ ]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        # __init__ 함수 내부에 nn.Linear을 선언하여 self.linear에 저장 
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.linear(x)
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)

        return y



GPU 사용하기 - Cuda 함수 

In [54]:
# GPU의 메모리 상에 텐서 생성 
x = torch.cuda.FloatTensor(2, 2)
print(x)

print('='*100)
# cuda 함수를 통해 CPU 메모리상에 선언된 텐서를 GPU로 복사 
x1 = torch.FloatTensor(2,2)
print(x1)
x2 = x1.cuda()
print(x2)

print('='*100)

# nn.Module 의 하위 클래스 객체에 적용 
import torch.nn as nn
layer = nn.Linear(2, 2)
layer.cuda(0)

### <주의>
### 텐서는 cuda 함수를 통해 원하는 디바이스로 복사가 되지만, 
### nn.Module 하위 클래스 객체는 "복사"가 아닌 "이동"이 된다

tensor([[2.3694e-38, 3.8177e-05],
        [0.0000e+00, 5.1569e-01]], device='cuda:0')
tensor([[1.4013e-45, 1.5549e-01],
        [1.4013e-45, 0.0000e+00]])
tensor([[1.4013e-45, 1.5549e-01],
        [1.4013e-45, 0.0000e+00]], device='cuda:0')


Linear(in_features=2, out_features=2, bias=True)

GPU 사용하기 - 서로 다른 장치 간 연산 

In [56]:
# 서로 다른 장치에 올라가 있는 텐서/nn.Module의 하위 클래스 객체끼리는 연산 불가 
# CPU와 GPU에 위치한 텐서들끼리도 연산 불가 
# GPU 0번과 GPU 1번 사이도 연산 불가 

x = torch.FloatTensor(2, 2)
x

print(x + x.cuda(0))


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

GPU 사용하기 - CPU 함수 

In [58]:
# GPU 메모리에 있는 텐서를 CPU 메모리로 복사해야 할 때 
x = torch.cuda.FloatTensor(2, 2) 
print(x)

x1 = x.cpu()
print(x1)


tensor([[0., 0.],
        [0., 0.]], device='cuda:0')
tensor([[0., 0.],
        [0., 0.]])


GPU 사용하기 - To함수 

In [74]:
cpu_device = torch.device('cpu')
gpu_device = torch.device('cuda:0')

# CPU
x = torch.FloatTensor(2, 2)
print(x)

# CPU -> GPU 
x1 = x.to(gpu_device)
print(x1)

# GPU -> CPU 
x2 = x1.to(cpu_device)
print(x2)

tensor([[       nan, 0.0000e+00],
        [5.6052e-45, 0.0000e+00]])
tensor([[       nan, 0.0000e+00],
        [5.6052e-45, 0.0000e+00]], device='cuda:0')
tensor([[       nan, 0.0000e+00],
        [5.6052e-45, 0.0000e+00]])


GPU 사용하기 - Device 속성 

In [78]:
# 텐서는 device 속성을 가지고 있어, 해당 텐서가 위치한 디바이스를 쉽게 파악 가능 
x = torch.cuda.FloatTensor(2, 2)
print(x.device)

# But, nn.Module의 하위 클래스 객체는 device 속성이 없다. 
# So, 아래와 같은 방법을 사용한다.

layer = nn.Linear(2, 2)
print(next(layer.parameters()).device)

cuda:0
cpu


# MSE Loss

In [80]:
# MSE Loss 구현 
def mse(x_hat, x):
    # |x_hat| = (batch_size, dim)
    # |x| = (batch_size, dim)
    y = ((x-x_hat)**2).mean()

    return y

x = torch.FloatTensor([[1,1],
                       [2,2]])
x_hat = torch.FloatTensor([[0,0],
                           [0,0]])

print(mse(x_hat, x))


tensor(2.5000)


torch.nn.functional 사용하기 

In [86]:
# 파이토치 내장 MSE 손실 함수 활용 
import torch.nn.functional as F
F.mse_loss(x_hat, x)

tensor(2.5000)

In [87]:
print(F.mse_loss(x_hat, x, reduction='sum'))
print(F.mse_loss(x_hat, x, reduction='none'))

tensor(10.)
tensor([[1., 1.],
        [4., 4.]])


torch.nn 사용하기 

In [88]:
import torch.nn as nn
mse_loss = nn.MSELoss()
mse_loss(x_hat, x)      

### torch.nn.functional 과 torch.nn 의 차이는 거의 없다 
### But, torch.nn은 nn.Module의 하위 클래스 내부에 선언하기에, 레이어의 하나처럼 취급 가능함

tensor(2.5000)

# 경사하강법 구현

In [111]:
target = torch.FloatTensor([[.1, .2, .3],
                            [.4, .5, .6],
                            [.7, .8, .9]])


In [112]:
x = torch.rand_like(target)
# This means the final scalar will be differentiate by x.
x.requires_grad = True
# You can get gradient of x, after differentiation.

x

tensor([[0.3744, 0.9964, 0.2138],
        [0.8847, 0.1022, 0.0541],
        [0.6321, 0.2298, 0.8967]], requires_grad=True)

In [113]:
loss = F.mse_loss(x, target)

loss

tensor(0.1931, grad_fn=<MseLossBackward>)

In [114]:
threshold = 1e-5
learning_rate = 1.
iter_cnt = 0

while loss > threshold:
    iter_cnt += 1
    
    loss.backward() # Calculate gradients.

    x = x - learning_rate * x.grad
    
    # You don't need to aware this now.
    x.detach_()
    x.requires_grad_(True)
    
    loss = F.mse_loss(x, target)
    
    print('%d-th Loss: %.4e' % (iter_cnt, loss))
    print(x)

1-th Loss: 1.1682e-01
tensor([[0.3134, 0.8195, 0.2330],
        [0.7770, 0.1906, 0.1754],
        [0.6472, 0.3565, 0.8974]], requires_grad=True)
2-th Loss: 7.0671e-02
tensor([[0.2660, 0.6818, 0.2479],
        [0.6932, 0.2593, 0.2697],
        [0.6589, 0.4550, 0.8980]], requires_grad=True)
3-th Loss: 4.2752e-02
tensor([[0.2291, 0.5747, 0.2595],
        [0.6280, 0.3128, 0.3431],
        [0.6681, 0.5317, 0.8985]], requires_grad=True)
4-th Loss: 2.5862e-02
tensor([[0.2004, 0.4915, 0.2685],
        [0.5774, 0.3544, 0.4002],
        [0.6752, 0.5913, 0.8988]], requires_grad=True)
5-th Loss: 1.5645e-02
tensor([[0.1781, 0.4267, 0.2755],
        [0.5380, 0.3868, 0.4446],
        [0.6807, 0.6377, 0.8991]], requires_grad=True)
6-th Loss: 9.4642e-03
tensor([[0.1607, 0.3763, 0.2809],
        [0.5073, 0.4119, 0.4791],
        [0.6850, 0.6738, 0.8993]], requires_grad=True)
7-th Loss: 5.7253e-03
tensor([[0.1472, 0.3371, 0.2852],
        [0.4835, 0.4315, 0.5060],
        [0.6883, 0.7018, 0.8994]], requi

파이토치 오토그래드 

In [132]:
# requires_grad=True 인 텐서는 연산 결과가 담긴 텐서도 속성값이 True를 갖게됨 
x = torch.FloatTensor([[1,2],
                       [3,4]]).requires_grad_(True)

x1 = x+2
print(x1)

x2 = x-2
print(x2)

x3 = x1*x2
print(x3)

y = x3.sum()
print(y)

y.backward() # 스칼라 값에 backward 함수를 호출하여 자동 미분을 수행

print(f'x.grad : {x.grad}')

x3.detach_() # 텐서를 메모리에서 해제 

tensor([[3., 4.],
        [5., 6.]], grad_fn=<AddBackward0>)
tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)
tensor([[-3.,  0.],
        [ 5., 12.]], grad_fn=<MulBackward0>)
tensor(14., grad_fn=<SumBackward0>)
x.grad : tensor([[2., 4.],
        [6., 8.]])


tensor([[-3.,  0.],
        [ 5., 12.]])